# Website A/B Testing - Lab

## Introduction

In this lab, you'll get another chance to practice your skills at conducting a full A/B test analysis. It will also be a chance to practice your data exploration and processing skills! The scenario you'll be investigating is data collected from the homepage of a music app page for audacity.

## Objectives

You will be able to:
* Analyze the data from a website A/B test to draw relevant conclusions
* Explore and analyze web action data

## Exploratory Analysis

Start by loading in the dataset stored in the file 'homepage_actions.csv'. Then conduct an exploratory analysis to get familiar with the data.

> Hints:
    * Start investigating the id column:
        * How many viewers also clicked?
        * Are there any anomalies with the data; did anyone click who didn't view?
        * Is there any overlap between the control and experiment groups? 
            * If so, how do you plan to account for this in your experimental design?

In [200]:
#Your code here
import pandas as pd
import numpy as np
import scipy.stats as stats

df = pd.read_csv('homepage_actions.csv')

click_ids = df[df['action'] == 'click']['id']
view_ids = df[df['action'] == 'view']['id']

df[(df['id'].isin(click_ids)) & (~df['id'].isin(view_ids))]

,timestamp,id,group,action


## Conduct a Statistical Test

Conduct a statistical test to determine whether the experimental homepage was more effective than that of the control group.

In [201]:
#Your code here
experiment = df['group'] == 'experiment'; control = df['group'] == 'control'

clicks = df['action'] == 'click'; views = df['action'] == 'views'

# get number of PURE viewers in the experiment group
exp_views = len(df[experiment & ~clicks]) # here we get the people who viewed, but we also still have people who viewed who also clicked
exp_clicks = len(df[experiment & clicks]) # here we get the people who clicked
exp_view_not_clicks = exp_views - exp_clicks # here we remove viewers who also clicked, to get people who ONLY viewed

# get number of PURE viewers in the control group
cont_views = len(df[control & ~clicks])
cont_clicks = len(df[control & clicks])
cont_view_not_clicks = cont_views - cont_clicks

observed = np.array([exp_clicks, exp_view_not_clicks]) 
expected = np.array([cont_clicks, cont_view_not_clicks])

_freq_table = pd.DataFrame(
    np.array([observed, expected]), 
    columns=['Clicked', 'Viewed'],
    index=['Observed', 'Expected']
)

chi2, p = stats.chisquare(f_obs=observed, f_exp=expected)

In [202]:
print('Frequency Table:\n')
print(_freq_table, '\n')
print('Statistics:\n')
print(f'''chi-square: {chi2}
p-value: {p}''')

Frequency Table:

          Clicked  Viewed
Observed      928    2068
Expected      932    2400 

Statistics:

chi-square: 45.943834048640916
p-value: 1.2169234221091101e-11


## Verifying Results

One sensible formulation of the data to answer the hypothesis test above would be to create a binary variable representing each individual in the experiment and control group. This binary variable would represent whether or not that individual clicked on the homepage; 1 for they did and 0 if they did not. 

The variance for the number of successes in a sample of a binomial variable with n observations is given by:

## $n\bullet p (1-p)$

Given this, perform 3 steps to verify the results of your statistical test:
1. Calculate the expected number of clicks for the experiment group, if it had the same click-through rate as that of the control group. 
2. Calculate the number of standard deviations that the actual number of clicks was from this estimate. 
3. Finally, calculate a p-value using the normal distribution based on this z-score.

### Step 1:
Calculate the expected number of clicks for the experiment group, if it had the same click-through rate as that of the control group. 

In [326]:
#Your code here
experiment_group = df[df['group'] == 'experiment'].copy()
control_group = df[df['group'] == 'control'].copy()

for group in [experiment_group, control_group]:
    group.loc[~(group['action'] == 'click'), 'action'] = 0
    group.loc[(group['action'] == 'click'), 'action'] = 1

num_clicks_cont = control_group['action'].sum()
everyone_cont = len(control_group.drop_duplicates(subset=['id'])) # every unique individual in control group

clickthrough_rate = num_clicks_cont / everyone_cont

everyone_exp = len(experiment_group.drop_duplicates(subset=['id'])) # every unique individual in experiment group
expected_num_clicks_exp = everyone_exp * clickthrough_rate

print(f'{expected_num_clicks_exp:,.0f} clicks')

838 clicks


### Step 2:
Calculate the number of standard deviations that the actual number of clicks was from this estimate.

In [337]:
#Your code here

actual_clicks_exp = experiment_group['action'].sum()

n_exp = everyone_exp
p = clickthrough_rate

distance = (actual_clicks_exp - expected_num_clicks_exp)
variance = n_exp * (p * (1-p))

stddevs_away =  distance / np.sqrt(variance)
stddevs_away

3.6625360854823588

### Step 3: 
Finally, calculate a p-value using the normal distribution based on this z-score.

In [342]:
# Your code here

# Since this is a binomial distribution, not a normal distribution, I used the binom test
p_value = stats.binom_test(actual_clicks_exp, n=everyone_exp, p=clickthrough_rate)
print(f'p_value: {p_value:,.5f}')

p_value: 0.00029


### Analysis:

Does this result roughly match that of the previous statistical test?

> Comment: **Yes. They roughly match in that they are both below an alpha of 0.05. This would make the both test results statistically significant, allowing us to reject the null hypothesis that the click-through rate for the experiment group is equal to the click-through rate for the control group.**

## Summary

In this lab, you continued to get more practice designing and conducting AB tests. This required additional work preprocessing and formulating the initial problem in a suitable manner. Additionally, you also saw how to verify results, strengthening your knowledge of binomial variables, and reviewing initial statistical concepts of the central limit theorem, standard deviation, z-scores, and their accompanying p-values.